# Components generator

### Parameters

In [1]:
import math 
from datetime import date
from IPython.display import display, Markdown, clear_output
import ipywidgets as widgets

In [12]:
# Define the list of words to select from
words = ['line', 'MVx', 'MVy', '']

# Define the initial value of FILE_NAME
REG_NAME = 'reg_line'

# Create the dropdown widget
word_select = widgets.Dropdown(
    options=words,
    value=words[0],
    description='Select a register:'
)

# Create the button widget
button = widgets.Button(description='Generate register file')

# Define a function to update the FILE_NAME variable
def update_filename(b):
    global REG_NAME
    REG_NAME = 'reg_{}'.format(word_select.value)
    print('Register ', REG_NAME, ' generated.')

# Attach the function to the button's on-click event
button.on_click(update_filename)

# Display the widgets
display(word_select)
display(button)


Dropdown(description='Select a register:', options=('line', 'MVx', 'MVy', ''), value='line')

Button(description='Generate register file', style=ButtonStyle())

Register  reg_MVx  generated.


In [14]:
REG_NAME

'reg_MVx'

In [4]:
button = widgets.Button(description='My Button')
out = widgets.Output()
def on_button_clicked(_):
      # "linking function with output"
      with out:
          # what happens when we press the button
          clear_output()
          print('Something happens!')
# linking button and function together using a button's method
button.on_click(on_button_clicked)
# displaying button and its output together
widgets.VBox([button,out])

In [5]:
# some options
a, b = 1, 5
# selecting global variables without underscores
global_variables = list(globals().keys())
funcs = []
for i in global_variables:
    if '_' not in i:
        funcs.append(i)
 
# creating menu with them   
global_vars = widgets.Dropdown(
    options=funcs)

# button, output, function and linkage
butt = widgets.Button(description='Print Variable')
outt = widgets.Output()
def on_butt_clicked(b):
    with outt:
        clear_output()
        print(type(globals()[global_vars.value]))
        print(globals()[global_vars.value])
        
butt.on_click(on_butt_clicked)
# display
widgets.VBox([global_vars,butt,outt])

In [6]:
#########################################
# Setup parameters
#########################################

N = 4                   # Input size (N x N)
TAPS = 6                # Number of taps
IN_BIT_WIDTH = 8        # Number of bits as input for each block in an input line
FILTERS = 15            # Number of filters (= number of outputs expected)
# OUTPUT_WIDTH = [11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11]         # Bitwidth for each output from interpolator
OUTPUT_WIDTH = 11       # One size for every output, as it is for my case


#########################################
# Enables writing new files
#########################################
WRITE_REG_BANK_FILE = True
WRITE_MUX_FILE = True
WRITE_REG_LINE_FILE = True

### Register bank as buffer

In [7]:
######################################################
# Variables calulation: Register bank as buffer
######################################################

NUM_OF_REGS = N + TAPS - 1
SEL_REG_WIDHT = str(math.ceil(math.log2(NUM_OF_REGS))-1)
EACH_OUT_WIDTH = str(FILTERS*OUTPUT_WIDTH-1)

# Loop to generate all possible resets values
RST_REG_BANK = []
for i in range(NUM_OF_REGS):
    RST_REG_BANK.append('REG_BANK['+str(i)+'] <= '+str(int(EACH_OUT_WIDTH)+1)+"'b0;")
RST_REG_BANK_CONCAT = '\n\t\t\t'.join(RST_REG_BANK)

# print(rst_reg_bank_concat)


In [8]:
###############################################
# Text writing: Register bank as buffer
###############################################

register_bank_text = '''/*------------------------------------------------------------------------------
 * File: reg_bank_buffer.v
 * Date generated: 2023-03-13
 * Date modified: '''+str(date.today())+'''
 * Author: Bruna Suemi Nagai
 * Description: A buffer to store each cycle output for the 2nd pass.
 *------------------------------------------------------------------------------ */

module reg_bank_buffer(
    CLK,
    RST_ASYNC_N,
    ADDR_SEL, 
    DATA_IN, 
    WRITE_EN, 
    READ_EN, 
    DATA_OUT
);


// ------------------------------------------
// IO declaration
// ------------------------------------------
	input CLK;                              // Clock
	input RST_ASYNC_N;						// Asynchronous reset
    input ['''+SEL_REG_WIDHT+''':0] ADDR_SEL;					// 4 bits to select between N+T-1=9 different registers
    input signed ['''+EACH_OUT_WIDTH+''':0] DATA_IN;			// Data to be written 
    input WRITE_EN;							// Enables writing
	input READ_EN;							// Enables reading
    output signed ['''+EACH_OUT_WIDTH+''':0] DATA_OUT;		    // Data read output
    
    
// ------------------------------------------
// Signals definitions
// ------------------------------------------
reg signed ['''+EACH_OUT_WIDTH+''':0] REG_BANK ['''+str(NUM_OF_REGS-1)+''':0];               	// Register bank with 9 registers of 8 bits each
reg signed ['''+EACH_OUT_WIDTH+''':0] REG_BANK_OUT; 					// Register for intermediate output data

assign DATA_OUT = REG_BANK_OUT;				// Intermediate output data to definetly output


// ------------------------------------------
// Sequential logic
// ------------------------------------------
always @(posedge CLK, negedge RST_ASYNC_N) begin
   if (!RST_ASYNC_N)
	begin
            '''+RST_REG_BANK_CONCAT+'''
	end
	
   else if (WRITE_EN) 		 					// If write enable is high
	begin
		REG_BANK[ADDR_SEL] <= DATA_IN; 			// Write data to the register at the specified address
	end 
	
	else if (READ_EN)
	begin
		REG_BANK_OUT <= REG_BANK[ADDR_SEL]; 	// Output data from the register at the specified address
	end
end

endmodule

'''
# print(register_bank_text)

if WRITE_REG_BANK_FILE:
    with open('reg_bank_buffer.v', 'w') as reg_bank_file:
        reg_bank_file.write(register_bank_text)

### Multiplexer

In [9]:
#########################################
# Variables setup: Mux generic 
#########################################
NUM_MUX_INPUTS = 2                      # Number of inputs 
NUM_MUX_OUTPUTS = 1                     # Number of outputs


#########################################
# Variables calulation
#########################################
SEL_MUX_WIDTH = math.ceil(math.log2(NUM_MUX_INPUTS))
EACH_OUT_WIDTH_MUX = FILTERS*OUTPUT_WIDTH


#########################################
# Loop to generate inputs declaration
#########################################
IN_MUX = []
IN_MUX_DECLARATION = []
for i in range(NUM_MUX_INPUTS):
    IN_MUX.append('IN'+str(i))
    IN_MUX_DECLARATION.append('input signed ['+str(EACH_OUT_WIDTH_MUX-1)+':0] IN'+str(i)+';')
IN_MUX_CONCAT = ',\n\t'.join(IN_MUX)
IN_MUX_ALWAYS_CONCAT = ' or '.join(IN_MUX)
IN_MUX_DECLARATION_CONCAT = '\n\t'.join(IN_MUX_DECLARATION)


#########################################
# Generating SEL declaration 
#########################################
if SEL_MUX_WIDTH > 1:
    SEL_DECLARATION = 'input unsigned ['+str(SEL_MUX_WIDTH-1)+':0] SEL;'
else:
    SEL_DECLARATION = 'input SEL;'


#########################################
# Loop to generate case
#########################################
FORMAT_OPTION = '#0'+str(SEL_MUX_WIDTH+2)+'b'
CASE_GENERATOR = []
for i in range(NUM_MUX_INPUTS):
    BINARY = str(format(i, FORMAT_OPTION))
    CASE_GENERATOR.append(str(SEL_MUX_WIDTH)+"'b"+BINARY[2:]+': OUT = IN'+str(i)+';')
CASE_GENERATOR_CONCAT = '\n\t\t'.join(CASE_GENERATOR)
# CASE_DEFAULT = str(EACH_OUT_WIDTH_MUX)+"'b"+format(0, '#0'+str(EACH_OUT_WIDTH_MUX+2)+'b')[2:]

# print(CASE_DEFAULT)
# print(CASE_GENERATOR_CONCAT)
# print(IN_MUX_CONCAT)
# print(IN_MUX_ALWAYS_CONCAT)
# print(IN_MUX_DECLARATION_CONCAT)
# print(SEL_DECLARATION)

In [10]:
###############################################
# Text writing: Mux generic
###############################################

mux_21_text = '''/*------------------------------------------------------------------------------
 * File: mux_'''+str(NUM_MUX_INPUTS)+str(NUM_MUX_OUTPUTS)+'''.v
 * Date generated: 2023-03-27
 * Date modified: '''+str(date.today())+'''
 * Author: Bruna Suemi Nagai
 * Description: Multiplexer '''+str(NUM_MUX_INPUTS)+':'+str(NUM_MUX_OUTPUTS)+'''
 *------------------------------------------------------------------------------ */

module mux_'''+str(NUM_MUX_INPUTS)+str(NUM_MUX_OUTPUTS)+'''(
	'''+IN_MUX_CONCAT+''',
	SEL, 
	OUT
);


// ------------------------------------------
// IO declaration
// ------------------------------------------
	'''+IN_MUX_DECLARATION_CONCAT+''' 
	'''+SEL_DECLARATION+''' 
	output reg signed ['''+str(EACH_OUT_WIDTH_MUX-1)+''':0] OUT;
	
	
// ------------------------------------------
// Logic
// ------------------------------------------
	always @ (SEL or '''+IN_MUX_ALWAYS_CONCAT+''')
	begin
  	case(SEL)
		'''+CASE_GENERATOR_CONCAT+'''
    	default: OUT = '''+str(EACH_OUT_WIDTH_MUX)+''''b0;
  	endcase
end
endmodule
'''
# print(mux_21_text)
if WRITE_MUX_FILE:
    with open('mux_'+str(NUM_MUX_INPUTS)+str(NUM_MUX_OUTPUTS)+'.v', 'w') as mux_file:
        mux_file.write(mux_21_text)

### All Registers 

In [11]:
#########################################
# Variables setup: Registers  
#########################################
REG_TYPE = 'line'

#########################################
# Variables calculations
#########################################
if 
DATA_IN_WIDTH = (N+TAPS-1)*IN_BIT_WIDTH                 # Bit width for the total input line

SyntaxError: invalid syntax (Temp/ipykernel_12188/133474621.py, line 9)

In [ ]:
###############################################
# Text writing: Mux generic
###############################################
reg_line_text = '''/*-----------------------------------------------------------------------------------
 * File: reg_line.v
 * Date generated: 2023-03-25
 * Date modified: '''+str(date.today())+'''
 * Author: Bruna Suemi Nagai
 * Description: A register to store the input line of padding + intergers from ref block.
 *----------------------------------------------------------------------------------- */

module reg_line(
    CLK,
    RST_ASYNC_N, 
    WRITE_EN,
    DATA_IN,  
    DATA_OUT
);


// ------------------------------------------
// IO declaration
// ------------------------------------------
	input CLK;                              // Clock
	input RST_ASYNC_N;						// Asynchronous reset
    input WRITE_EN;							// Enables writing
    input signed ['''+str(DATA_IN_WIDTH-1)+''':0] DATA_IN;			// Data to be written 
    output reg signed ['''+str(DATA_IN_WIDTH-1)+''':0] DATA_OUT;	    // Data read output
    

// ------------------------------------------
// Sequential logic
// ------------------------------------------
always @(posedge CLK, negedge RST_ASYNC_N) begin
   if (!RST_ASYNC_N)                        // If rst async is low
	begin
            DATA_OUT <= '''+str(DATA_IN_WIDTH)+''''b0;
	end
	
   	else if (WRITE_EN) 		 			    // If write enable is high
	begin
		DATA_OUT <= DATA_IN; 			    // Write data to the register at the specified address
	end 
end

endmodule

'''
# print(reg_line_text)

if WRITE_REG_LINE_FILE:
    with open('reg_line.v', 'w') as reg_line_file:
        reg_line_file.write(reg_line_text)

: 

: 